In [1]:
import Model
import tensorflow as tf
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
import os
import re
import hashlib
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  
import SpeechGenerator
import librosa
#from keras import losses

#from extractMFCC import computeFeatures, computeFeatures1
#from addNoise import addNoise
print(tf.__version__)

Using TensorFlow backend.


1.14.0


In [2]:
# There is a problem with the maximum lenght of a string then We save the Dataset root folder as a variable
dataset_dir = "Dataset/"

In [3]:
testWAVs = pd.read_csv(dataset_dir + 'testing_list.txt', sep=" ", header=None)[0].tolist()
valWAVs  = pd.read_csv(dataset_dir + 'validation_list.txt', sep=" ", header=None)[0].tolist()
"""
for idx, item in enumerate(testWAVs):
    testWAVs[idx] = "Dataset/" + testWAVs[idx]

for idx, item in enumerate(valWAVs):
    valWAVs[idx] = "Dataset/" + valWAVs[idx]
"""

'\nfor idx, item in enumerate(testWAVs):\n    testWAVs[idx] = "Dataset/" + testWAVs[idx]\n\nfor idx, item in enumerate(valWAVs):\n    valWAVs[idx] = "Dataset/" + valWAVs[idx]\n'

In [4]:
DictCategs = {'nine' : 1, 'yes' : 2, 'no' : 3, 'up' : 4, 'down' : 5, 'left' : 6, 'right' : 7, 'on' : 8, 'off' : 9, 
              'stop' : 10, 'go' : 11, 'zero' : 12, 'one' : 13, 'two' : 14, 'three' : 15, 'four' : 16, 'five' : 17, 
              'six' : 18, 'seven' : 19, 'eight' : 20, 'backward':0, 'bed':0, 'bird':0, 'cat':0, 'dog':0, 'follow':0, 
              'forward':0, 'happy':0, 'house':0, 'learn':0, 'marvin':0, 'sheila':0, 'tree':0, 'visual':0, 'wow':0 }
nCategs = 21

In [5]:
allWAVs  = []
for root, dirs, files in os.walk('Dataset/'):
    for f in files:
        if (root != dataset_dir + "_background_noise_") and (f.endswith('.wav')):
            path = root + "/" + f
            #print(path)
            path = path[len(dataset_dir):]
            #print(path)
            allWAVs.append(path)
    """
    print(f for f in files if f.endswith('.wav'))
    if root != dataset_dir + "_background_noise_":
        allWAVs += [root+'/'+ f for f in files if f.endswith('.wav')]
    """
trainWAVs = list(set(allWAVs) - set(valWAVs) - set(testWAVs))

In [6]:
print(len(trainWAVs))
print(len(valWAVs))
print(len(testWAVs))

84843
9981
11005


In [7]:
def _getFileCategory(file, catDict):
    # Receives a file with name <cat>/<filename> and returns an integer that is catDict[cat]
    categ = os.path.basename(os.path.dirname(file))
    return catDict.get(categ, 0)

In [8]:
#get categories
testWAVlabels = [_getFileCategory(f, DictCategs) for f in testWAVs]
valWAVlabels = [_getFileCategory(f, DictCategs) for f in valWAVs]
trainWAVlabels = [_getFileCategory(f, DictCategs) for f in trainWAVs]

In [9]:
print(len(trainWAVlabels))
print(len(valWAVlabels))
print(len(testWAVlabels))

84843
9981
11005


In [10]:
"""def load_and_preprocess_data(file_name):
    # Required by tensorflow (strings are passed as bytes)
    # tf.strings.as_string(...)
    file_name = file_name.numpy()
    file_name = file_name.decode("utf-8") 
    #print(file_name)
    #print(type(file_name))

    # Load data
    feats = computeFeatures(file_name)
    # Normalize
    mean = np.mean(feats, axis = 0)
    stv = np.std(feats, axis = 0)
    diff = np.subtract(feats, mean)
    feats = np.divide(diff, stv)

    return feats.astype(np.float32)

# example:

feats = load_and_preprocess_data(trainWAVs[25])
feats = np.transpose(feats)

plt.figure(figsize=(17,6))
plt.pcolormesh(feats)

plt.title('Spectrogram visualization')
plt.ylabel('Frequency')
plt.xlabel('Time')

plt.show()
print(trainWAVlabels[25])
"""

'def load_and_preprocess_data(file_name):\n    # Required by tensorflow (strings are passed as bytes)\n    # tf.strings.as_string(...)\n    file_name = file_name.numpy()\n    file_name = file_name.decode("utf-8") \n    #print(file_name)\n    #print(type(file_name))\n\n    # Load data\n    feats = computeFeatures(file_name)\n    # Normalize\n    mean = np.mean(feats, axis = 0)\n    stv = np.std(feats, axis = 0)\n    diff = np.subtract(feats, mean)\n    feats = np.divide(diff, stv)\n\n    return feats.astype(np.float32)\n\n# example:\n\nfeats = load_and_preprocess_data(trainWAVs[25])\nfeats = np.transpose(feats)\n\nplt.figure(figsize=(17,6))\nplt.pcolormesh(feats)\n\nplt.title(\'Spectrogram visualization\')\nplt.ylabel(\'Frequency\')\nplt.xlabel(\'Time\')\n\nplt.show()\nprint(trainWAVlabels[25])\n'

In [11]:
"""def create_dataset(file_names, labels, batch_size, shuffle, cache_file=None):
    
    # Create a Dataset object
    dataset = tf.data.Dataset.from_tensor_slices((file_names, labels))
    
    # Map the load_and_preprocess_data function
    py_func = lambda file_name, label: (tf.py_function(load_and_preprocess_data, [file_name], Tout=tf.float32), label)
    dataset = dataset.map(py_func, num_parallel_calls=os.cpu_count())
    
    # Cache dataset
    if cache_file:
        dataset = dataset.cache(cache_file)
    
    # Shuffle    
    if shuffle:
        dataset = dataset.shuffle(len(file_names))
        
    # Repeat the dataset indefinitely
    dataset = dataset.repeat()
    
    # Correct input shape for the network
    dataset = dataset.map(lambda data, label: (tf.expand_dims(data, 1), label))
    
    # Batch
    dataset = dataset.batch(batch_size=batch_size)
    
    # Prefetch
    dataset = dataset.prefetch(buffer_size=1)
    
    return dataset
"""

'def create_dataset(file_names, labels, batch_size, shuffle, cache_file=None):\n    \n    # Create a Dataset object\n    dataset = tf.data.Dataset.from_tensor_slices((file_names, labels))\n    \n    # Map the load_and_preprocess_data function\n    py_func = lambda file_name, label: (tf.py_function(load_and_preprocess_data, [file_name], Tout=tf.float32), label)\n    dataset = dataset.map(py_func, num_parallel_calls=os.cpu_count())\n    \n    # Cache dataset\n    if cache_file:\n        dataset = dataset.cache(cache_file)\n    \n    # Shuffle    \n    if shuffle:\n        dataset = dataset.shuffle(len(file_names))\n        \n    # Repeat the dataset indefinitely\n    dataset = dataset.repeat()\n    \n    # Correct input shape for the network\n    dataset = dataset.map(lambda data, label: (tf.expand_dims(data, 1), label))\n    \n    # Batch\n    dataset = dataset.batch(batch_size=batch_size)\n    \n    # Prefetch\n    dataset = dataset.prefetch(buffer_size=1)\n    \n    return dataset\n'

In [12]:
"""batch_size = 32
train_dataset = create_dataset(trainWAVs, 
                               trainWAVlabels, 
                               batch_size=batch_size, 
                               shuffle=True,
                               cache_file='train_cache')

val_dataset = create_dataset(valWAVs, 
                             valWAVlabels,
                             batch_size=batch_size, 
                             shuffle=False,
                             cache_file='val_cache')

train_steps = int(np.ceil(len(trainWAVs)/batch_size))
val_steps = int(np.ceil(len(valWAVs)/batch_size))
"""

"batch_size = 32\ntrain_dataset = create_dataset(trainWAVs, \n                               trainWAVlabels, \n                               batch_size=batch_size, \n                               shuffle=True,\n                               cache_file='train_cache')\n\nval_dataset = create_dataset(valWAVs, \n                             valWAVlabels,\n                             batch_size=batch_size, \n                             shuffle=False,\n                             cache_file='val_cache')\n\ntrain_steps = int(np.ceil(len(trainWAVs)/batch_size))\nval_steps = int(np.ceil(len(valWAVs)/batch_size))\n"

In [13]:
# Transoform the dataset in numpy array and load 
train = np.array(trainWAVs, dtype = object)

print("Loading of the train set:")
for i in range(len(trainWAVs)):
    # Print the progress 
    if (i % 5000) == 0:
        print(str(i) + '/' + str(len(trainWAVs)))
    
    # If the file is not already present, we create the numpy version 
    if (not os.path.isfile(dataset_dir + "/" + trainWAVs[i] + '.npy')):
        y, sr = librosa.load(dataset_dir + "/" + trainWAVs[i], sr = 16000)
        np.save(dataset_dir + "/" + trainWAVs[i] + '.npy', y)
    
    # We load the path to numpy array in a vector 
    train[i] = trainWAVs[i] + '.npy'
    
print(str(i) + '/' + str(len(trainWAVs)))

Loading of the train set:
0/84843
5000/84843
10000/84843
15000/84843
20000/84843
25000/84843
30000/84843
35000/84843
40000/84843
45000/84843
50000/84843
55000/84843
60000/84843
65000/84843
70000/84843
75000/84843
80000/84843
84842/84843


In [14]:
# Do the same thing for the validation and the test set
val = np.array(valWAVs, dtype = object)

print("Loading of the validation set:")
for i in range(len(valWAVs)):
    # Print the progress 
    if (i % 5000) == 0:
        print(str(i) + '/' + str(len(valWAVs)))
    
    # If the file is not already present, we create the numpy version 
    if (not os.path.isfile(dataset_dir + "/" + valWAVs[i] + '.npy')):
        y, sr = librosa.load(dataset_dir + "/" + valWAVs[i], sr = 16000)
        np.save(dataset_dir + "/" + valWAVs[i] + '.npy', y)
    
    # We load the path to numpy array in a vector 
    val[i] = valWAVs[i] + '.npy'
    
print(str(i) + '/' + str(len(valWAVs)))

test = np.array(testWAVs, dtype = object)

print("Loading of the test set:")
for i in range(len(testWAVs)):
    # Print the progress 
    if (i % 5000) == 0:
        print(str(i) + '/' + str(len(testWAVs)))
    
    # If the file is not already present, we create the numpy version 
    if (not os.path.isfile(dataset_dir + "/" + testWAVs[i] + '.npy')):
        y, sr = librosa.load(dataset_dir + "/" + testWAVs[i], sr = 16000)
        np.save(dataset_dir + "/" + testWAVs[i] + '.npy', y)
    
    # We load the path to numpy array in a vector 
    test[i] = testWAVs[i] + '.npy' 
    
print(str(i) + '/' + str(len(testWAVs)))

Loading of the validation set:
0/9981
5000/9981
9980/9981
Loading of the test set:
0/11005
5000/11005
10000/11005
11004/11005


In [15]:
#test on the length
print(str(len(trainWAVs)) + " = " + str(len(train)))
print(str(len(valWAVs)) + " = " + str(len(val)))
print(str(len(testWAVs)) + " = " + str(len(test)))

#test on labels 
for i in range(0, 25):
    print("file: " + train[i] + " - label: " + str(trainWAVlabels[i]))
    print("file: " + trainWAVs[i] + " - label: " + str(trainWAVlabels[i]))

84843 = 84843
9981 = 9981
11005 = 11005
file: eight/c86d4fd4_nohash_0.wav.npy - label: 20
file: eight/c86d4fd4_nohash_0.wav - label: 20
file: one/d926bb17_nohash_0.wav.npy - label: 13
file: one/d926bb17_nohash_0.wav - label: 13
file: nine/f575faf3_nohash_1.wav.npy - label: 1
file: nine/f575faf3_nohash_1.wav - label: 1
file: three/fbb2f6cc_nohash_1.wav.npy - label: 15
file: three/fbb2f6cc_nohash_1.wav - label: 15
file: tree/cdee383b_nohash_0.wav.npy - label: 0
file: tree/cdee383b_nohash_0.wav - label: 0
file: marvin/2f813234_nohash_0.wav.npy - label: 0
file: marvin/2f813234_nohash_0.wav - label: 0
file: two/e9b61425_nohash_0.wav.npy - label: 14
file: two/e9b61425_nohash_0.wav - label: 14
file: eight/61e2f74f_nohash_1.wav.npy - label: 20
file: eight/61e2f74f_nohash_1.wav - label: 20
file: house/3d6bee47_nohash_1.wav.npy - label: 0
file: house/3d6bee47_nohash_1.wav - label: 0
file: backward/a7acbbeb_nohash_0.wav.npy - label: 0
file: backward/a7acbbeb_nohash_0.wav - label: 0
file: no/7cb5c

In [16]:
trainGen = SpeechGenerator.SpeechGen(train, trainWAVlabels, dataset_dir, batch_size = 32, shuffle = True)
valGen = SpeechGenerator.SpeechGen(val, valWAVlabels, dataset_dir, shuffle = False)    

In [17]:
model = Model.AttentionModel(21, 99, 60, use_GRU = True, dropout = 0.0, activation = 'relu')
"""
decayed_lr = tf.train.exponential_decay(0.0000001,
                                        0.8, 50,
                                        0.95, staircase=True)

adam = tf.train.AdamOptimizer(learning_rate=decayed_lr,  
                                beta1=0.9,
                                beta2=0.999,
                                epsilon=1e-07,
                                use_locking=False,
                                name='Adam')
"""
#adam = tf.train.AdamOptimizer(learning_rate=0.00000001,  
#                                beta1=0.9,
#                                beta2=0.999,
#                                epsilon=1e-07,
#                                use_locking=False,
#                                name='Adam')

#model.compile(optimizer=adam,
#              loss= tf.losses.softmax_cross_entropy,
#              metrics=[keras.metrics.sparse_categorical_accuracy, ])

model.compile(optimizer='adam', loss=['sparse_categorical_crossentropy'], metrics=['sparse_categorical_accuracy'])
model.summary()

W0625 17:37:03.459053 139676877416192 deprecation_wrapper.py:119] From /home/jupyter/.local/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0625 17:37:03.475083 139676877416192 deprecation_wrapper.py:119] From /home/jupyter/.local/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0625 17:37:03.618856 139676877416192 deprecation_wrapper.py:119] From /home/jupyter/.local/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:1521: The name tf.log is deprecated. Please use tf.math.log instead.

W0625 17:37:03.645856 139676877416192 deprecation_wrapper.py:119] From /home/jupyter/.local/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0625 17:37:03.672529 139676

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 16000)        0                                            
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 1, 16000)     0           input_1[0][0]                    
__________________________________________________________________________________________________
mel_stft (Melspectrogram)       (None, 80, 125, 1)   1091664     reshape_1[0][0]                  
__________________________________________________________________________________________________
normalization2d_1 (Normalizatio (None, 80, 125, 1)   0           mel_stft[0][0]                   
__________________________________________________________________________________________________
permute_1 

In [18]:
import math
def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.4
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop,  
            math.floor((1+epoch)/epochs_drop))
    
    if (lrate < 4e-5):
        lrate = 4e-5
      
    print('Changing learning rate to {}'.format(lrate))
    return lrate
lrate = LearningRateScheduler(step_decay)


earlystopper = EarlyStopping(monitor='val_sparse_categorical_accuracy', patience=10, verbose=1)
checkpointer = ModelCheckpoint('model-attRNN.h5', monitor='val_sparse_categorical_accuracy', verbose=1, save_best_only=True)

In [19]:
num_epochs = 3
history = model.fit_generator(trainGen, validation_data = valGen, epochs = num_epochs, callbacks = [earlystopper, checkpointer, lrate])

# Save the model
model.save('Model/my_model_sparse-loss_dropout-0_1.h5')
print(history.history.keys())
# Plot loss
plt.figure()
plt.plot(history.history['loss'], label='Train loss')
plt.plot(history.history['val_loss'], label='Val loss')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Loss')

# Plot accuracy
plt.figure()
plt.plot(history.history['sparse_categorical_accuracy'], label='Train acc')
plt.plot(history.history['val_sparse_categorical_accuracy'], label='Val acc')
plt.legend()

plt.xlabel('Epoch')
plt.ylabel('Accuracy')

W0625 17:37:19.556581 139676877416192 deprecation.py:323] From /home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/3
Changing learning rate to 0.001
2651/2651 [==============================] - 1065s 402ms/step - loss: 0.9805 - sparse_categorical_accuracy: 0.7101 - val_loss: nan - val_sparse_categorical_accuracy: 0.8740

Epoch 00001: val_sparse_categorical_accuracy improved from -inf to 0.87399, saving model to model-attRNN.h5
Epoch 2/3
Changing learning rate to 0.001
2651/2651 [==============================] - 1076s 406ms/step - loss: 0.3824 - sparse_categorical_accuracy: 0.8851 - val_loss: nan - val_sparse_categorical_accuracy: 0.8955

Epoch 00002: val_sparse_categorical_accuracy improved from 0.87399 to 0.89546, saving model to model-attRNN.h5
Epoch 3/3
Changing learning rate to 0.001
 365/2651 [===>..........................] - ETA: 14:44 - loss: nan - sparse_categorical_accuracy: 0.2673

KeyboardInterrupt: 

In [21]:
testGen = SpeechGenerator.SpeechGen(test, testWAVlabels, dataset_dir, shuffle=False)

testEval = model.evaluate_generator(testGen, verbose = 1)

print("loss: " + str(testEval[0]))
print("acc: " + str(testEval[1]))

171/171 [==============================] - 47s 277ms/step
loss: nan
acc: 0.2580409356725146
